Importar librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Cargue de dataframe

In [3]:
df_cat = pd.read_excel('/content/drive/MyDrive/proyectos IA/proyecto_llm/compilado.xlsx')

In [4]:
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71429 entries, 0 to 71428
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Recurso                     71429 non-null  object
 1   Estado de actividad         71429 non-null  object
 2   Fecha de Generación         57219 non-null  object
 3   Descripción de la orden     71402 non-null  object
 4   Orden de trabajo            71429 non-null  object
 5   Zona de trabajo             71241 non-null  object
 6   Motivo de No Realizado      14905 non-null  object
 7   Comentarios No Realizado    14903 non-null  object
 8   Comentarios de Cancelación  28836 non-null  object
 9   Motivo de Cancelación       29034 non-null  object
 10  Observaciones               27371 non-null  object
dtypes: object(11)
memory usage: 6.0+ MB


Validar datos nulos

In [5]:
df_cat.isnull().sum()

,0
Recurso,0
Estado de actividad,0
Fecha de Generación,14210
Descripción de la orden,27
Orden de trabajo,0
Zona de trabajo,188
Motivo de No Realizado,56524
Comentarios No Realizado,56526
Comentarios de Cancelación,42593
Motivo de Cancelación,42395


Pre procesamiento de Comentarios y Motivo de cancelación

In [6]:
# comentarios en minúsculas
df_cat['Comentarios de Cancelación'] = df_cat['Comentarios de Cancelación'].str.lower()
df_cat['Motivo de Cancelación'] = df_cat['Motivo de Cancelación'].str.lower()


In [7]:
eliminar_comentarios = ['manualm', 'creado manual', 'cargue manual', 'inc duplicado', 'aviso duplicado', 'incidente duplicado.', 'duplicidad', 'duplicada', 'duplicado']
eliminar_motivos = ['Cancelacion por Sistema Externo']

# Filtrar el DataFrame para eliminar filas que contengan los parámetros
df_cat = df_cat[~df_cat['Comentarios de Cancelación'].str.contains('|'.join(eliminar_comentarios), na=False)]
# Filtrar el DataFrame para eliminar filas que contengan los parámetros
df_cat = df_cat[~df_cat['Motivo de Cancelación'].str.contains('|'.join(eliminar_motivos), na=False)]

In [8]:
# Reemplazar NaN por cadenas vacías para evitar concatenar NaN
df_cat['motivo'] = (df_cat['Motivo de No Realizado'].fillna('') + ' ' + df_cat['Motivo de Cancelación'].fillna('')).astype(str)
df_cat['resolucion'] = (df_cat['Comentarios de Cancelación'].fillna('') + ' ' + df_cat['Comentarios No Realizado'].fillna('')).astype(str)

In [9]:
#pasar texto a minúsculas y eliminar espacios en blanco
df_cat['motivo'] = df_cat['motivo'].str.lower().str.strip()
df_cat['resolucion'] = df_cat['resolucion'].str.lower().str.strip()

# Listas de palabras clave para reasignar motivo
motivos_amarre = ['cliente de tolima', 'cliente de valle', 'dirección doble tolima', 'dirección doble valle']
sector_a = ['sector a']
atendido = ['atendida con otro tiquete', 'atendida con otro aviso']

# Condiciones y reasignación de la columna 'motivo'
df_cat['motivo'] = df_cat['motivo'].apply(lambda x: 'mal amarre' if any(keyword in str(x) for keyword in motivos_amarre)
                                            else ('sector a' if any(keyword in str(x) for keyword in sector_a)
                                                  else ('atendido con otro incidente' if any(keyword in str(x) for keyword in atendido)
                                                        else x)))


Eliminar duplicados por Estado de actividad y concatenar resultados

In [10]:
# Filtrar solo los finalizados y eliminar duplicados
finalizados = df_cat[df_cat['Estado de actividad'] == 'finalizada'].drop_duplicates()

# Filtrar canceladas y eliminar duplicados
canceladas = df_cat[df_cat['Estado de actividad'] == 'cancelada'].drop_duplicates()

# Filtrar no realizadas y eliminar duplicados
no_realizadas = df_cat[df_cat['Estado de actividad'] == 'no realizado'].drop_duplicates()

# Unir todos los DataFrames filtrados
df_duplicated = pd.concat([finalizados, canceladas, no_realizadas]).drop_duplicates()

Eliminar duplicados en las diferentes combinaciones de Estado de actividad priorizando finalizadas y concatenar

In [11]:
#Filtrar por "finalizada" y "no realizado" y eliminar duplicados dando prioridad a finalizada
finalizada_no_realizado = df_duplicated[(df_duplicated['Estado de actividad'].isin(['finalizada', 'no realizado']))]
finalizada_no_realizado = finalizada_no_realizado.sort_values('Estado de actividad', ascending=False)
finalizada_no_realizado = finalizada_no_realizado.drop_duplicates(subset=['Orden de trabajo'], keep='first')

#Filtrar por "finalizada" y "cancelada" y eliminar duplicados dando prioridad a finalizada
finalizada_cancelada = df_duplicated[(df_duplicated['Estado de actividad'].isin(['finalizada', 'cancelada']))]
finalizada_cancelada = finalizada_cancelada.sort_values('Estado de actividad', ascending=False)
finalizada_cancelada = finalizada_cancelada.drop_duplicates(subset=['Orden de trabajo'], keep='first')

#Filtrar por "cancelada" y "no realizado" y eliminar duplicados dando prioridad a cancelada
cancelada_no_realizado = df_duplicated[(df_duplicated['Estado de actividad'].isin(['cancelada', 'no realizado']))]
cancelada_no_realizado = cancelada_no_realizado.sort_values('Estado de actividad', ascending=False)
cancelada_no_realizado = cancelada_no_realizado.drop_duplicates(subset=['Orden de trabajo'], keep='first')

df = pd.concat([finalizada_no_realizado, finalizada_cancelada, cancelada_no_realizado]).drop_duplicates()

Limpieza de datos y asignación de categorización de salida

In [12]:
# Limpiar los datos: convertir a minúsculas y eliminar espacios extra
df['motivo'] = df['motivo'].str.lower().str.strip()
df['resolucion'] = df['resolucion'].str.lower().str.strip()

#Definir salida
ejecutar = ['condiciones climaticas', 'normalización bt', 'daño interno del cliente', 'servicio confirmado con cliente']

df['out'] = df['motivo'].apply(
    lambda x: 'ejecutar' if (x == '' or pd.isna(x) or any(keyword in x for keyword in ejecutar)) else 'pendiente'
)
df['out'].unique()

array(['pendiente', 'ejecutar'], dtype=object)

In [13]:
# Condición 1
condicion1 = (
    df['Descripción de la orden'].str.contains("acometida", case=False, na=False) &
    df['resolucion'].str.contains("lac", case=False, na=False) &
    ~df['resolucion'].str.contains("instalación|instalacion", case=False, na=False)
)
df.loc[condicion1, 'out'] = "cancelar"

# Condición 2
condicion2 = (
    (df['motivo'] == 'servicio normal') &
    df['resolucion'].str.contains("trabajo pendiente|conector", case=False, na=False)
)
df.loc[condicion2, 'out'] = "ejecutar"

# Condición 3
condicion3 = (
    df['Descripción de la orden'].str.contains("acometida|alumbrado|desconexión", case=False, na=False) &
    (df['motivo'] == 'servicio normal')
)
df.loc[condicion3, 'out'] = "cancelar"

# Condición 4
condicion4 = (
    (df['motivo'] == 'servicio normal') &
    df['resolucion'].str.contains("servicio normal ok", case=False, na=False)
)
df.loc[condicion4, 'out'] = "cancelar"

# Condición 5
condicion5 = (
    df['Descripción de la orden'].str.contains("acometida", case=False, na=False) &
    df['Descripción de la orden'].str.contains("red media tensión|rama|poste desplomado|poste caido|poste caído|descolgada|riesgo", case=False, na=False) &
    (df['motivo'] == 'servicio normal') &
    df['resolucion'].str.contains("normalizan|apoya|encuentra normal|energía normal|energia normal|trabajo pendiente|conector", case=False, na=False)
)
df.loc[condicion5, 'out'] = "ejecutar"

# Condición 6
condicion6 = (
    df['Descripción de la orden'].str.contains("acometida|guadua|sector b", case=False, na=False) &
    (df['motivo'] == 'sector a') &
    df['resolucion'].str.contains("levantamiento|peligro|instala", case=False, na=False)
)
df.loc[condicion6, 'out'] = "ejecutar"

# Condición 7
condicion7 = (
    (df['motivo'] == 'sector a') &
    df['Descripción de la orden'].str.contains("guadua|sector b", case=False, na=False)
) | df['resolucion'].str.contains("levantamiento|peligro|instalación|instalacion", case=False, na=False)
df.loc[condicion7, 'out'] = "ejecutar"

# Condición 8
condicion8 = (
    df['Descripción de la orden'].str.contains("acometida", case=False, na=False) &
    df['motivo'].str.contains("atendido con otro|atendido con otra|dificil acceso|daño interno del cliente|falta logística|mal amarre|normalización bt|orden publico alterado|otro operador de red|servicio confirmado con cliente", case=False, na=False)
)
df.loc[condicion8, 'out'] = "ejecutar"

# Condición 9
condicion9 = (
    df['Descripción de la orden'].str.contains("acometida|red media tensión|rama|poste desplomado", case=False, na=False) &
    df['motivo'].str.contains("atendido con otro|atendido con otra|dificil acceso|daño interno del cliente|falta logística|mal amarre|normalización bt|orden publico alterado|otro operador de red|servicio confirmado con cliente", case=False, na=False)
)
df.loc[condicion9, 'out'] = "ejecutar"



#Condición 10
condicion10 = (
    df['Descripción de la orden'].str.contains("medidor", case=False, na=False) &
    df['resolucion'].str.contains("alto consumo", case=False, na=False)
)

# Asignar
df.loc[condicion10, 'out'] = "cancelar"



#Condición 11
condicion11 = (
    df['Descripción de la orden'].str.contains("medidor", case=False, na=False) &
    df['motivo'].str.contains("atendido con otro|cumplimiento retie|daño interno del cliente|dificl acceso|falta logistica|mal amarre|normalizacion bt|servicio confirmado con cliente|usuario no permite", case=False, na=False)
)

# Asignar
df.loc[condicion11, 'out'] = "ejecutar"


#Condición 12
condicion12 = (
    df['Descripción de la orden'].str.contains("medidor|alto consumo", case=False, na=False) &
    (df['motivo'] == 'sector a') &
    df['resolucion'].str.contains("reconexión", case=False, na=False)
)

# Asignar
df.loc[condicion12, 'out'] = "cancelar"



#Condición 13
condicion13 = (
    df['Descripción de la orden'].str.contains("medidor", case=False, na=False) &
    (df['motivo'] == 'servicio normal') &
    df['resolucion'].str.contains("fuga|maniobra", case=False, na=False)
)

# Asignar
df.loc[condicion13, 'out'] = "ejecutar"


#Condición 14
condicion14 = (
    df['Descripción de la orden'].str.contains("medidor", case=False, na=False) &
    (df['motivo'] == 'servicio normal')
)

# Asignar
df.loc[condicion14, 'out'] = "cancelar"


#Condición 15
condicion15 = (
    df['Descripción de la orden'].str.contains("medidor", case=False, na=False) &
    (df['motivo'] == 'solicitud de cancelación') &
    df['resolucion'].str.contains("sector a", case=False, na=False)
)

# Asignar
df.loc[condicion15, 'out'] = "cancelar"


#Condición 16
condicion16 = (
    df['Descripción de la orden'].str.contains("medidor", case=False, na=False) &
    (df['motivo'] == 'solicitud de cancelación')
)

# Asignar
df.loc[condicion16, 'out'] = "cancelar"


#Condición 17
condicion17 = (
    df['Descripción de la orden'].str.contains("alumbrado", case=False, na=False) &
    df['motivo'].str.contains("atendido con otro incidente|atendido con otro tiquete|daño interno del cliente|normalizacion bt|servicio confirmado con cliente", case=False, na=False)
)

# Asignar
df.loc[condicion17, 'out'] = "ejecutar"


#Condición 18
condicion18 = (
    df['Descripción de la orden'].str.contains("alumbrado|riesgo|peligro", case=False, na=False) &
    df['motivo'].str.contains("falta información", case=False, na=False) &
    df['resolucion'].str.contains("apoyo|linea viva", case=False, na=False)
)

# Asignar
df.loc[condicion18, 'out'] = "ejecutar"


#Condición 19
condicion19 = (
    df['Descripción de la orden'].str.contains("alumbrado", case=False, na=False) &
    df['motivo'].str.contains("falta logistica", case=False, na=False)
)

# Asignar
df.loc[condicion19, 'out'] = "cancelar"


#Condición 20
condicion20 = (
    df['Descripción de la orden'].str.contains("alumbrado", case=False, na=False) &
    df['motivo'].str.contains("otro operador de red", case=False, na=False)
)

# Asignar
df.loc[condicion20, 'out'] = "cancelar"


#Condición 21
condicion21 = (
    df['Descripción de la orden'].str.contains("alumbrado", case=False, na=False) &
    df['motivo'].str.contains("sector a", case=False, na=False)
)

# Asignar
df.loc[condicion21, 'out'] = "cancelar"



#Condición 22
condicion22 = (
    df['Descripción de la orden'].str.contains("alumbrado", case=False, na=False) &
    df['motivo'].str.contains("servicio normal|solicitud de cancelación", case=False, na=False)
)

# Asignar
df.loc[condicion22, 'out'] = "cancelar"


#Condición 23
condicion23 = (
    df['Descripción de la orden'].str.contains("antena", case=False, na=False) &
    df['motivo'].str.contains("atendido con otro incidente|daño interno del cliente|dificl acceso|falta logistica|falta información|normalización bt|mal amarre|servicio confirmado con cliente|solicitud de cancelación|usuario no permite", case=False, na=False)
)

# Asignar
df.loc[condicion23, 'out'] = "ejecutar"


#Condición 24
condicion24 = (
    df['Descripción de la orden'].str.contains("antena", case=False, na=False) &
    df['motivo'].str.contains("servicio normal|operador telefonía celular", case=False, na=False)
)

# Asignar
df.loc[condicion24, 'out'] = "cancelar"


#Condición 25
condicion25 = (
    df['Descripción de la orden'].str.contains("torre de telec", case=False, na=False)
)

# Asignar
df.loc[condicion25, 'out'] = "cancelar"



In [14]:
df = df[~df['motivo'].isin(['cancelacion por sistema externo'])]

#Condición 26
condicion26 = (
    df['motivo'].str.contains("atendido con otro incidente|atendido con otro tiquete|dificil acceso|mal amarre|tlm normalizada|interventoria", case=False, na=False)
)

# Asignar
df.loc[condicion26, 'out'] = "cancelar"


#Condición 27
condicion27 = (
    df['Descripción de la orden'].str.contains("internet", case=False, na=False)
)

# Asignar
df.loc[condicion27, 'out'] = "cancelar"

Eliminar datos sin categorización de salida

In [15]:
df = df[~df['out'].isin(['pendiente'])]

Validar distribución de Dataset

In [16]:
df['out'].value_counts().to_frame().sort_index()

,count
out,
cancelar,13261
ejecutar,41079


Generar dataset para modelo LLM

In [20]:
df.to_csv('/content/drive/MyDrive/proyectos IA/proyecto_llm/df.csv', index=False)

In [21]:
df.to_excel('/content/drive/MyDrive/proyectos IA/proyecto_llm/df.xlsx', index=False)